In [1]:
%reset -f
# import the necessary packages
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K

import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
# from pyimagesearch.lenet import LeNet
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os

# initialize the number of epochs to train for, initial learning rate,
# and batch size
epochs = 100
batch_size = 32
monitor='val_loss'
es_patience=7
rlr_patience=3
img_width, img_height = 60,60

In [2]:
def get_dataset(dataset='./insectrec/created_data/impy_crops_export/'):
    # initialize the data and labels
    print(" loading images...")
    data = []
    labels = []

    # grab the image paths and randomly shuffle them
    imagePaths = sorted(list(paths.list_images(dataset)))
    random.seed(42)
    random.shuffle(imagePaths)

    # loop over the input images
    for imagePath in imagePaths:
        # load the image, pre-process it, and store it in the data list
        image = cv2.imread(imagePath)
        image = cv2.resize(image, (img_width, img_height))
        image = img_to_array(image)
        data.append(image)

        # extract the class label from the image path and update the
        # labels list
        label = imagePath.split(os.path.sep)[-2]
        labels.append(label)

    # scale the raw pixel intensities to the range [0, 1]
    data = np.array(data, dtype="float") / 255.0

    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    labels = le.fit_transform(labels)

    # partition the data into training and testing splits using 75% of
    # the data for training and the remaining 25% for testing
    (trainX, testX, trainY, testY) = train_test_split(data,
        labels, test_size=0.2, random_state=42)
    return trainX, testX, trainY, testY, labels

_, testX, _, testY, _ = get_dataset(dataset='./insectrec/created_data/impy_crops_export/')
trainX, _, trainY, _, labels = get_dataset(dataset='./insectrec/created_data/images_augmented/')

# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=6)
testY = to_categorical(testY, num_classes=6)

 loading images...
 loading images...


In [7]:
# testY.shape

In [8]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
    height_shift_range=0.1, zoom_range=0.3,
    horizontal_flip=True, vertical_flip=True,fill_mode="nearest")

In [9]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau, CSVLogger

log_dir = './insectrec/created_data/logs/'
top_weights_path = './insectrec/created_data/weights/'

callbacks_list = [ModelCheckpoint(monitor =  monitor,
                                  filepath =  top_weights_path,
                                  save_best_only = True,
                                  save_weights_only = False,
                                  verbose = 1),
                  EarlyStopping(monitor =  monitor,
                                patience =  es_patience,
                                verbose = 1),
                  ReduceLROnPlateau(monitor =  monitor,
                                    factor = 0.1,
                                    patience =  rlr_patience,
                                    verbose = 1),
                # CSVLogger(filename =  logfile),
                  TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)]


In [12]:
base_model = DenseNet121(include_top=True, weights=None, 
                         input_shape=(img_width,img_height,3))
# add a global spatial average pooling layer
x = base_model.output
# x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(256, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(6, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)



In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
FH = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // batch_size,
    epochs=epochs, verbose=1, max_queue_size=100, workers=16, callbacks=callbacks_list)

Epoch 1/100
 424/1910 [=====>........................] - ETA: 25:38 - loss: 1.5330 - accuracy: 0.3841

In [8]:
base_model = DenseNet121(include_top=False, weights='imagenet', 
                         input_shape=(img_width,img_height,3))
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(256, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(6, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# train the model on the new data for a few epochs
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // batch_size,
    epochs=5, verbose=1)

Epoch 1/5
439/439 [==============================] - 418s 953ms/step - loss: 1.4824 - accuracy: 0.4256 - val_loss: 1.6483 - val_accuracy: 0.3299
Epoch 2/5
439/439 [==============================] - 429s 978ms/step - loss: 1.3466 - accuracy: 0.4812 - val_loss: 1.5725 - val_accuracy: 0.3507
Epoch 3/5
439/439 [==============================] - 421s 958ms/step - loss: 1.3114 - accuracy: 0.4903 - val_loss: 1.5725 - val_accuracy: 0.3720
Epoch 4/5
439/439 [==============================] - 425s 969ms/step - loss: 1.2864 - accuracy: 0.4989 - val_loss: 1.6181 - val_accuracy: 0.3567
Epoch 5/5
439/439 [==============================] - 419s 955ms/step - loss: 1.2537 - accuracy: 0.5227 - val_loss: 1.5375 - val_accuracy: 0.3857


In [ ]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

In [11]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first L layers and unfreeze the rest:
L = 313
for layer in model.layers[:L]:
   layer.trainable = False
for layer in model.layers[L:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
optimizer= SGD(lr=1e-4, momentum=0.9)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
FH = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // batch_size,
    epochs=epochs, verbose=1, max_queue_size=40, workers=1, callbacks=callbacks_list)

Epoch 1/100
438/439 [============================>.] - ETA: 0s - loss: 1.2460 - accuracy: 0.5206
Epoch 00001: val_loss improved from inf to 1.70719, saving model to ./insectrec/created_data/weights/
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./insectrec/created_data/weights/assets
439/439 [==============================] - 471s 1s/step - loss: 1.2453 - accuracy: 0.5208 - val_loss: 1.7072 - val_accuracy: 0.3882
Epoch 2/100
438/439 [============================>.] - ETA: 0s - loss: 1.1642 - accuracy: 0.5574
Epoch 00002: val_loss did not improve from 1.70719
439/439 [==============================] - 454s 1s/step - loss: 1.1641 - accuracy: 0.5574 - val_loss: 1.8167 - val_accuracy: 0.3771
Epoch 3/100
438/439 [============================>.] - ETA: 0s - loss: 1.1157 - accuracy: 0.5753
Epoch 00003: val_loss improved from 1.70719 to 1.54660, saving model to ./insectrec/created_data/weights/
INFO:tensorflow:Assets written

In [ ]:
FH